In [ ]:
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.pulse import PulseSequence, Frame, Port
from braket.device_schema.rigetti.rigetti_device_parameters_v1 import RigettiDeviceParameters

# Set up the Rigetti Ankaa-3 device (public ARN, safe to include)
device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")

small_angle_rx = 0.05
small_angle_cry = 0.05

# Build the quantum circuit as used in the experiment
circuit = Circuit()
circuit.h(38)
circuit.rx(38, small_angle_rx)
circuit.h(39)
circuit.cnot(38, 39).ry(39, small_angle_cry / 2)
circuit.cnot(38, 39).ry(39, -small_angle_cry / 2)

# Pulse sequence configuration for decoherence control
port_38 = Port(port_id='q38_QRRx2Channel-0', dt=1e-9, properties=None)
port_39 = Port(port_id='q39_QRRx2Channel-0', dt=1e-9, properties=None)

frame_38 = Frame(
    frame_id='Transmon_38_readout_rx',
    port=port_38,
    frequency=7296975715.952661
)

frame_39 = Frame(
    frame_id='Transmon_39_readout_rx',
    port=port_39,
    frequency=7461668250.311123
)

pulse_sequence = PulseSequence().delay(
    qubits_or_frames=[frame_38, frame_39],
    duration=40e-6
)

device_parameters = RigettiDeviceParameters(
    paradigmParameters={
        "qubitCount": 82,
        "pulseSequence": pulse_sequence.to_ir()
    }
)

# NOTE: Replace 'your-anonymized-bucket' with your own S3 bucket for results storage
s3_folder = ("your-anonymized-bucket", "tasks")

task = device.run(
    circuit,
    shots=100,
    device_parameters=device_parameters,
    s3_destination_folder=s3_folder
)

result = task.result()

print("Measurement Counts:", result.measurement_counts)


# Analysis Workflow

## Instructions

1. Run the quantum circuit 30 times using the provided code above.
2. After completing all runs, execute the following code block to retrieve the latest 30 quantum tasks and their results from AWS Braket.
3. Copy the printed output (Task ID, Measurement Counts, Task Start, Task End) and paste it at the top of the analysis notebook.
4. Proceed with the analysis using these results and the corresponding EEG data.

**Note:**  
- Make sure your AWS credentials are properly configured.  
- Replace any sensitive information (such as S3 bucket names) with anonymized values before sharing or publishing the notebook.


In [ ]:
import boto3
from braket.aws import AwsQuantumTask

# Create an AWS Braket client (specify the region)
braket_client = boto3.client('braket', region_name='us-west-1')

# Retrieve the most recent 30 quantum tasks (no filters)
response = braket_client.search_quantum_tasks(
    filters=[],
    maxResults=30
)

tasks = response.get('quantumTasks', [])

# Collect and display task information
tasks_info = []

for task in tasks:
    task_id = task['quantumTaskArn'].split('/')[-1]
    task_obj = AwsQuantumTask(task['quantumTaskArn'])
    result = task_obj.result()
    metadata = task_obj.metadata()

    tasks_info.append({
        "task_id": task_id,
        "measurement_counts": result.measurement_counts,
        "start_time": metadata.get('createdAt', 'N/A'),
        "end_time": metadata.get('endedAt', 'N/A')
    })

# Sort tasks by start time
tasks_info.sort(key=lambda x: x['start_time'])

# Print the results (copy and paste this output into your analysis notebook)
for info in tasks_info:
    print(f"Task ID: {info['task_id']}")
    print("Measurement Counts:", info["measurement_counts"])
    print("Task Start:", info["start_time"])
    print("Task End:", info["end_time"])
    print("-" * 40)
